In [14]:
import faiss
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image
from deepface import DeepFace
from scipy.spatial.distance import cosine
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

In [15]:
target_size = (224, 224)
batch_size = 8
images_path = glob.glob("../my_images/*")

In [16]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(target_size[0], target_size[1], 3))

feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

def extract_feature(img_path):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    features = feature_extractor.predict(img_array)
    features = features.flatten()

    return features

In [17]:
image_features = []
for img_path in images_path:
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    extracted_vector = extract_feature(img_path).flatten()
    image_features.append(extracted_vector)

image_features = np.array(image_features)

1/1 [==============================] - 0s 49ms/step


In [18]:
index = faiss.IndexFlatL2(image_features.shape[1])
index.add(image_features)

def find_closest(img_path):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    extracted_vector = feature_extractor.predict(img_array).flatten()
    extracted_vector = np.array([extracted_vector])
    _, result_search = index.search(extracted_vector, 1)

    return images_path[result_search[0][0]]

In [19]:
def is_it_me(img_path):
    extracted_vector = extract_feature(img_path)
    for my_img_vector in image_features:
        similarity = 1 - cosine(my_img_vector, extracted_vector)
        print(similarity)
        if similarity > 0.7:
            return True
    return False
    
print("This is you Abolfazl!") if is_it_me("../test_images/test.jpg") else print("This is not you!")

1/1 [==============================] - 0s 36ms/step
0.3930576845367937
0.4854797107840122
0.487245122698749
0.41090313984078364
0.374826283070961
0.3241338844692815
0.4544144279021234
0.5315107752612834
0.29831701127478827
0.545681315401576
0.6743856779351279
0.5308361271589825
0.7645760439904199
This is you Abolfazl!


In [20]:
embeddings = []
for my_img in images_path:
    try:
        embedding = DeepFace.represent(img_path=my_img, model_name="Facenet")
    except:
        print("Face has not detected!")
    embeddings.append(embedding[0]["embedding"])
embeddings

Face has not detected!
Face has not detected!


[[-0.7500391006469727,
  -0.3846389055252075,
  0.08184540271759033,
  -0.8064464330673218,
  0.13383492827415466,
  -1.0634981393814087,
  0.041198115795850754,
  -0.06108103320002556,
  -0.04884278029203415,
  -0.11853694915771484,
  0.6458025574684143,
  0.17972704768180847,
  -0.057056013494729996,
  -0.18707206845283508,
  -0.22959601879119873,
  0.16899681091308594,
  0.41416579484939575,
  -0.28971320390701294,
  0.13254381716251373,
  -0.8151159882545471,
  -0.44326627254486084,
  -0.06004538759589195,
  -0.15222206711769104,
  0.09598483145236969,
  -0.13463075459003448,
  0.2782926559448242,
  0.9808633327484131,
  0.018987290561199188,
  -0.2653173804283142,
  -0.5206995010375977,
  -0.5775431394577026,
  -0.23363398015499115,
  0.0734107494354248,
  0.030268847942352295,
  0.7312006950378418,
  0.5212160348892212,
  0.772777259349823,
  0.009694032371044159,
  0.5591195821762085,
  -0.47084811329841614,
  -0.36975932121276855,
  -0.033127039670944214,
  -0.02892912924289703

In [21]:
def is_it_me(img):
    embedding = DeepFace.represent("../test_images/test.jpg", model_name="Facenet")
    embedding_vector = embedding[0]["embedding"]
    fa = embedding[0]["facial_area"]
    x, y, w, h = fa["x"], fa["y"], fa["w"], fa["h"]
    for em in embeddings:
        similarity = 1 - cosine(em, embedding_vector)
        if similarity > 0.7:
            return embedding[0], True, (x, y, w, h)
    return None, False, None

In [22]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Abolfazl Detector")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    cv2.imshow("Abolfazl Detector", frame)
    
    if cv2.waitKey(1) & 0xFF == 13:
        embedding, result, face_info = is_it_me(frame)

        if result:
            x, y, w, h = face_info

            text = "This is you, Abolfazl!" if result else print("This is not you.")
            color = (0, 255, 0) if result else (0, 0, 255)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 2, cv2.LINE_AA)
            
            cv2.imshow("Abolfazl Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()